In [3]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /Users/omarleosamman/Documents/GitHub/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [4]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import os
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [5]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-16 22:54:15,757 INFO: Initializing external client
2025-11-16 22:54:15,764 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 22:54:17,422 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1286332


'{"country": "spain", "city": "madrid", "street": "castellana", "aqicn_url": "https://api.waqi.info/feed/@3231", "latitude": 40.42, "longitude": -3.7}'

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [6]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=2,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [7]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,13.0,spain,madrid,castellana,2025-11-16,https://api.waqi.info/feed/@3231


In [8]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 172.0+ bytes


## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [9]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 40.5°N -3.75°E
Elevation 667.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-16,13.00,0.1,16.923828,240.708740,madrid
1,2025-11-17,12.05,0.1,5.991594,57.264786,madrid
2,2025-11-18,10.75,0.0,2.741678,203.198624,madrid
3,2025-11-19,9.00,0.0,1.835647,101.309898,madrid
4,2025-11-20,7.80,0.1,11.183201,303.178528,madrid
5,2025-11-21,5.75,0.0,14.689588,323.972534,madrid
6,2025-11-22,5.70,0.0,4.024922,296.564972,madrid


In [10]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 352.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [ ]:
history_df = air_quality_fg.read()
history_df["date"] = pd.to_datetime(history_df["date"], utc=True).dt.tz_convert(None)

#Add correct date to today's data
aq_today_df["date"] = pd.to_datetime(aq_today_df["date"], utc=True).dt.tz_convert(None)

#Final list where we will store processed rows for each sensor
processed_rows = []

#Group today's data by sensor
for (country, city, street), df_today_sensor in aq_today_df.groupby(["country", "city", "street"]):

    #Filtrate the historical data ONLY for that sensor
    hist_sensor = history_df[
        (history_df["country"] == country) &
        (history_df["city"] == city) &
        (history_df["street"] == street)
    ].sort_values("date")

    #Take the last 3 days from history
    last_days_sensor = hist_sensor.tail(3)

    #Concatenate window + new data
    df_window = pd.concat([last_days_sensor, df_today_sensor], ignore_index=True)

    #Make sure date format is correct
    df_window["date"] = pd.to_datetime(df_window["date"])

    #Recalculate lags & rolling ONLY in this window
    df_window = util.add_lags_and_rolling(df_window)

    #Feature weekend
    df_window["is_weekend"] = (df_window["date"].dt.weekday >= 5).astype(int)

    #Extract the processed row for today
    df_today_processed = df_window[df_window["date"] == pd.to_datetime(today)].copy()

    processed_rows.append(df_today_processed)

#Insert to feature group
air_quality_fg.insert(df_all_today)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.90s) 


In [25]:
last_days_df = (
    air_quality_fg.read()
    .sort_values("date")
    .tail(3)   # solo las 3 últimas observaciones existentes
)
df_window = pd.concat([last_days_df, aq_today_df], ignore_index=True)
df_window["date"] = pd.to_datetime(df_window["date"], utc=True).dt.tz_convert(None)
df_window["date"] = pd.to_datetime(df_window["date"])
df_window = util.add_lags_and_rolling(df_window)
df_window['is_weekend'] = (df_window['date'].dt.weekday >= 5).astype(int)
df_today_processed = df_window[df_window["date"] == pd.to_datetime(today)].copy()



Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.09s) 


In [24]:
df_today_processed

,date,pm25,country,city,street,url,pm25_lag1,pm25_lag2,pm25_lag3,pm25_roll3,is_weekend
3,2025-11-16,13.0,spain,madrid,castellana,https://api.waqi.info/feed/@3231,29.0,31.0,45.0,24.333333,1


In [9]:
# Insert new data
air_quality_fg.insert(aq_today_df)

2025-11-15 16:14:17,118 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286332/fs/1273957/fg/1703404


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286332/jobs/named/air_quality_1_offline_fg_materialization/executions


(Job('air_quality_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747649
         }
       },
       "result": {
         "observed_value": 12.999999046325684,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-15T10:14:17.000118Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "succe

In [10]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-15 16:14:31,305 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1286332/fs/1273957/fg/1703405


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1286332/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-15 16:14:48,919 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-15 16:14:52,092 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-15 16:17:15,109 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-15 16:17:15,266 INFO: Waiting for log aggregation to finish.
2025-11-15 16:17:23,866 INFO: Execution finished successfully.


(Job('weather_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 747651
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-15T10:14:31.000304Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
